In [7]:
import numpy as np
# importing decimal module to control significant figures
import decimal

In [8]:
def toDecimal(array) :
    # flatten the array convert floats to decimal and reshape to original shape
    originalShape = array.shape
    array = array.flatten().astype(object) 
    for i in range(array.size) :
        array[i] = decimal.Decimal(str(array[i]))
    array = array.reshape(originalShape)   
    return array


def intializeContext(significantFigs,rounding) :
    #Intialize the context with specified signFigs and rounding/chopping
    decimal.getcontext().prec = significantFigs 
    if( rounding ) :
        decimal.getcontext().rounding = decimal.ROUND_HALF_UP
    else :
        decimal.getcontext().rounding = decimal.ROUND_DOWN 
        
def toFloats (array) :
    #Convert the decimals back to float
    originalShape = array.shape
    array = array.flatten().astype(object)
    for i in range(array.size) :
        array[i] = float(array[i]) 
    array = array.reshape(originalShape)    
    return array            
    

In [9]:
def isDiagonalyDominant(A) :
    #Check if Matrix A is diagonaly dominant 
    dim = A.shape
    # Flags to check system
    at_least_one_strictly_greater = None 
    if(dim[0] != dim[1]) : raise Exception("Matrix must be square")
    for i in range(dim[0]) :
        nonDiagonalSum = 0 
        for j in range(dim[0]) :
            if(j==i) : continue 
            nonDiagonalSum += abs(A[i][j]) 
        if(A[i][i] > nonDiagonalSum) :
            at_least_one_strictly_greater = True
        elif(A[i][i] == nonDiagonalSum) :
            if(at_least_one_strictly_greater == None) : at_least_one_strictly_greater = False
        else :
            return False    
    return at_least_one_strictly_greater            
            

In [10]:
# GaussSeidel with relaxation for improved convergence
# the parameter n can be removed as we can get the no of vars from the npArray
def GaussSeidel_noNorm(A,b,n,x,maxIterations,ErrorTolerance,relax , significantFigs = 7 , rounding = True) :
    # Setting up signifcant figs and rounding/chopping
    intializeContext(significantFigs,rounding)
    
    #converting floats to decimals 
    A = toDecimal(A)
    b = toDecimal(b)
    x = toDecimal(x)
    relax = decimal.Decimal(str(relax))
    
    #List to hold iteration details
    iteration_details = []
    #Check for diagonaly dominant matrix
    if(isDiagonalyDominant(A) ) :
        isDominant = 'The matrix is diagonaly dominant'
    else :
        isDominant = 'The matrix is not diagonaly dominant'    
    # Calculating first iteration before applying relaxation   
    var_steps = [] 
    for i in range(n) :
        computation_terms = [f"{b[i]}"]  # start with b_i
        sum = b[i]
        for j in range(n) :
            if(i==j) :
                continue
            sum -= A[i][j] * x[j]
            computation_terms.append(f" - ({A[i][j]} * {x[j]})")
        formula_str = "".join(computation_terms)
        x[i] = sum/A[i][i]
        var_steps.append(f"x{i+1} = ({formula_str} / {A[i][i]}) = {x[i]}")
    details = {
            'iteration' : 1,
            'xNew' : toFloats(x),
            'maxError' : '_',
            'steps' : var_steps
        }     
    iteration_details.append(details)
    iteration = 2
    # Loop until convergence or max iterations reached 
    while (True) :
        belowTolerance = True
        maxError = 0
        var_steps = []
        for i in range(n) :
            oldX = x[i]
            sum = b[i]
            computation_terms = [f"{b[i]}"]  # for formula string
            for j in range(n) :
                if(i==j) :
                    continue
                sum -= A[i][j] * x[j]
                computation_terms.append(f" - ({A[i][j]} * {x[j]})")
            formula_str = "".join(computation_terms)
            x[i] = relax*sum/A[i][i] + (1-relax)*oldX
            if (x[i] != 0) :
                estimatedError = abs(float((x[i]-oldX)/x[i])) * 100
                estimatedError = float(estimatedError)
                if(estimatedError > ErrorTolerance):
                    belowTolerance = False
                maxError = max(maxError, estimatedError)
            full_formula = (
                f"x{i+1} = {relax}*(({''.join(computation_terms)}) / {A[i][i]})"
                f" + (1-{relax})*{oldX} = {x[i]}"
            )    
            var_steps.append(full_formula)
        details = {
            'iteration' : iteration,
            'xNew' : toFloats(x),
            'maxError' : float(maxError),
            'steps' : var_steps
        }            
        iteration+=1
        iteration_details.append(details)
        if(belowTolerance or iteration >= maxIterations) :
            break
    lines = [isDominant]
    for d in iteration_details :
        lines.append(f"\n──────────── Iteration {d['iteration']} ────────────")
        for step in d['steps']:
            lines.append(step)
        lines.append(f"max error = {d['maxError']}")   
    return toFloats(x), lines

In [11]:
# Example linear system
A = np.array([[10, -1, 2, 0],
              [-1, 11, -1, 3],
              [2, -1, 10, -1],
              [0, 3, -1, 8]], dtype=float)

b = np.array([6, 25, -11, 15], dtype=float)

# Initial guess
x0 = np.zeros(len(b))

# Run Gauss-Seidel with relaxation
solution, details = GaussSeidel_noNorm(A, b, 4, x0, maxIterations=50, ErrorTolerance=1e-6, relax=0.8)

print("Solution:", solution)
print("\nIteration Details:")
print("\n".join(details))
# for d in details:
#     max_error = d['maxError']
#     # Ensure maxError is a float, else print as-is
#     if isinstance(max_error, (float, int)):
#         max_error_str = f"{max_error:.6f}"
#     else:
#         max_error_str = str(max_error)
#     print(f"Iteration {d['iteration']}: x = {d['xNew']}, maxError = {max_error_str}")


Solution: [1.0 2.0 -1.0 1.0]

Iteration Details:
The matrix is diagonaly dominant

──────────── Iteration 1 ────────────
x1 = (6.0 - (-1.0 * 0.0) - (2.0 * 0.0) - (0.0 * 0.0) / 10.0) = 0.6
x2 = (25.0 - (-1.0 * 0.6) - (-1.0 * 0.0) - (3.0 * 0.0) / 11.0) = 2.327273
x3 = (-11.0 - (2.0 * 0.6) - (-1.0 * 2.327273) - (-1.0 * 0.0) / 10.0) = -0.9872727
x4 = (15.0 - (0.0 * 0.6) - (3.0 * 2.327273) - (-1.0 * -0.9872727) / 8.0) = 0.8788635
max error = _

──────────── Iteration 2 ────────────
x1 = 0.8*((6.0 - (-1.0 * 2.327273) - (2.0 * -0.9872727) - (0.0 * 0.8788635)) / 10.0) + (1-0.8)*0.6 = 0.9441456
x2 = 0.8*((25.0 - (-1.0 * 0.9441456) - (-1.0 * -0.9872727) - (3.0 * 0.8788635)) / 11.0) + (1-0.8)*2.327273 = 2.088749
x3 = 0.8*((-11.0 - (2.0 * 0.9441456) - (-1.0 * 2.088749) - (-1.0 * 0.8788635)) / 10.0) + (1-0.8)*-0.9872727 = -0.9911087
x4 = 0.8*((15.0 - (0.0 * 0.9441456) - (3.0 * 2.088749) - (-1.0 * -0.9911087)) / 8.0) + (1-0.8)*0.8788635 = 0.9500371
max error = 36.45048

──────────── Iteration 3 ────

In [12]:
A = np.array([[3, -0.1, -0.2],[0.1,7,-0.3], [0.3,-0.2,10]], dtype=float)
b = np.array([7.85, -19.3, 71.4], dtype=float)
solution, details = GaussSeidel_noNorm(A, b, 3, np.zeros(3), maxIterations=25, ErrorTolerance=1e-6, relax=1.0)
print("Solution:", solution)
print("\nIteration Details:")
print("\n".join(details))

Solution: [3.0 -2.5 7.0]

Iteration Details:
The matrix is diagonaly dominant

──────────── Iteration 1 ────────────
x1 = (7.85 - (-0.1 * 0.0) - (-0.2 * 0.0) / 3.0) = 2.616667
x2 = (-19.3 - (0.1 * 2.616667) - (-0.3 * 0.0) / 7.0) = -2.794524
x3 = (71.4 - (0.3 * 2.616667) - (-0.2 * -2.794524) / 10.0) = 7.00561
max error = _

──────────── Iteration 2 ────────────
x1 = 1.0*((7.85 - (-0.1 * -2.794524) - (-0.2 * 7.00561)) / 3.0) + (1-1.0)*2.616667 = 2.990557
x2 = 1.0*((-19.3 - (0.1 * 2.990557) - (-0.3 * 7.00561)) / 7.0) + (1-1.0)*-2.794524 = -2.499626
x3 = 1.0*((71.4 - (0.3 * 2.990557) - (-0.2 * -2.499626)) / 10.0) + (1-1.0)*7.00561 = 7.000290
max error = 12.502350000000002

──────────── Iteration 3 ────────────
x1 = 1.0*((7.85 - (-0.1 * -2.499626) - (-0.2 * 7.000290)) / 3.0) + (1-1.0)*2.990557 = 3.000032
x2 = 1.0*((-19.3 - (0.1 * 3.000032) - (-0.3 * 7.000290)) / 7.0) + (1-1.0)*-2.499626 = -2.499987
x3 = 1.0*((71.4 - (0.3 * 3.000032) - (-0.2 * -2.499987)) / 10.0) + (1-1.0)*7.000290 = 6.99999